In [0]:
from google.colab import files
import pandas as pd

In [8]:
train = pd.read_csv( "labeledTrainData.tsv", header=0, 
 delimiter="\t", quoting=3 )

test = pd.read_csv( "testData.tsv", header=0, delimiter="\t", quoting=3 )
unlabeled_train = pd.read_csv( "unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3 )

# Verify the number of reviews that were read (100,000 in total)
print (f'Read {train["review"].size} labeled train reviews, {test["review"].size} labeled test reviews',
         f'and {unlabeled_train["review"].size} unlabeled reviews\n')

Read 25000 labeled train reviews, 25000 labeled test reviews and 50000 unlabeled reviews



In [0]:
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist( review, remove_stopwords=False ):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review).get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    #
    # 5. Return a list of words
    return(words)

In [10]:
import nltk.data
nltk.download('punkt')

# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( review_to_wordlist( raw_sentence))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
sentences = []  # Initialize an empty list of sentences

print ("Parsing sentences from training set")
for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

print ("Parsing sentences from unlabeled set")
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences from unlabeled set


/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.6/dist-packages/bs4/__init__.py:336: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behin

In [12]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',\
    level=logging.INFO)

# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec

2019-07-18 12:56:14,905 : INFO : 'pattern' package not found; tag filters are not available for English


In [13]:
print ("Training model...")
model = word2vec.Word2Vec(sentences, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "300features_40minwords_10context"
model.save(model_name)

2019-07-18 12:56:16,807 : INFO : collecting all words and their counts
2019-07-18 12:56:16,809 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-18 12:56:16,877 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2019-07-18 12:56:16,945 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model...


2019-07-18 12:56:17,013 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2019-07-18 12:56:17,089 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2019-07-18 12:56:17,156 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2019-07-18 12:56:17,224 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2019-07-18 12:56:17,287 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2019-07-18 12:56:17,356 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2019-07-18 12:56:17,422 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2019-07-18 12:56:17,491 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2019-07-18 12:56:17,565 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

In [0]:
upload = files.download(model_name)

KeyboardInterrupt: ignored

In [14]:
model.doesnt_match("man woman child kitchen".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'kitchen'

In [15]:
model.doesnt_match("france england germany berlin".split())

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'berlin'

In [0]:
import warnings
warnings.filterwarnings('ignore')

In [17]:
model.doesnt_match("paris berlin london austria".split())

'austria'

In [18]:
model.most_similar("man")

[('woman', 0.6315004825592041),
 ('lady', 0.5923056602478027),
 ('lad', 0.5679600238800049),
 ('millionaire', 0.5272843241691589),
 ('farmer', 0.5254068374633789),
 ('men', 0.5238955616950989),
 ('guy', 0.5229156613349915),
 ('monk', 0.519939661026001),
 ('soldier', 0.5136591792106628),
 ('person', 0.5123767256736755)]

In [19]:
model.most_similar("queen")

[('princess', 0.664714515209198),
 ('victoria', 0.620487630367279),
 ('bride', 0.6029039025306702),
 ('belle', 0.5976051688194275),
 ('mistress', 0.5760812759399414),
 ('matriarch', 0.5749611258506775),
 ('regina', 0.5722713470458984),
 ('goddess', 0.5695858597755432),
 ('mary', 0.5658347010612488),
 ('eva', 0.5638195276260376)]

In [0]:
model.most_similar("awful")

[('terrible', 0.7623046040534973),
 ('atrocious', 0.7368501424789429),
 ('horrible', 0.7337960004806519),
 ('abysmal', 0.721423327922821),
 ('dreadful', 0.7122504711151123),
 ('horrendous', 0.710689902305603),
 ('horrid', 0.6825776100158691),
 ('appalling', 0.6761379837989807),
 ('amateurish', 0.6340696811676025),
 ('lousy', 0.6154699921607971)]

In [0]:
model["flower"]

array([ 3.85456420e-02, -1.14445211e-02, -2.35223752e-02,  2.67233662e-02,
        7.46695371e-03, -8.95465687e-02, -5.14674932e-03,  5.23242764e-02,
        4.32693847e-02,  2.30202116e-02,  7.75928125e-02, -3.13363969e-02,
        6.35997429e-02, -1.67516936e-02, -6.67837448e-03, -8.27001259e-02,
        9.18907374e-02, -7.14611039e-02,  4.70008105e-02, -7.65618868e-03,
       -4.50852094e-03,  2.58687008e-02,  2.34368555e-02,  2.54384484e-02,
        5.47079965e-02, -2.37053093e-02, -2.61992421e-02, -9.71635208e-02,
       -6.64924011e-02, -1.67393982e-02, -7.33891129e-02, -2.75098113e-03,
       -9.91160497e-02,  1.46651501e-02, -6.85635433e-02,  5.34661636e-02,
       -7.07636774e-02,  1.07741371e-01, -3.78822088e-02,  8.72443318e-02,
        3.62220742e-02,  2.25024130e-02,  6.91870600e-02,  2.74810046e-02,
        8.60247314e-02, -5.63533651e-03,  1.19310915e-01, -7.36588016e-02,
       -4.36340384e-02, -3.52767967e-02,  4.49224412e-02,  1.05292588e-01,
       -7.19942898e-02, -

In [0]:
import numpy as np

def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.wv.vocab)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
          print (f"Review {counter} of {len(reviews)}")
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model,num_features)
       #
       # Increment the counter
       counter = counter + 1
    return reviewFeatureVecs

In [23]:
import nltk
nltk.download('stopwords')

train = pd.read_csv( "labeledTrainData.tsv", header=0, delimiter="\t")

clean_train_reviews = []
for review in train["review"]:
    clean_train_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

print ("Creating average feature vecs for test reviews")
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append( review_to_wordlist( review, \
        remove_stopwords=True ))

testDataVecs = getAvgFeatureVecs( clean_test_reviews, model, num_features )

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000
Creating average feature vecs for test reviews
Review 0 of 25000
Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 140

In [24]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier( n_estimators = 100 )

print ("Fitting a random forest to labeled training data...")
forest = forest.fit( trainDataVecs, train["sentiment"] )

# Test & extract results 
result = forest.predict( testDataVecs )

# Write the test results 
output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
output.to_csv( "Word2Vec_AverageVectors.csv", index=False, quoting=3 )

Fitting a random forest to labeled training data...


In [0]:
files.download('Word2Vec_AverageVectors.csv')

In [26]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster
word_vectors = np.array([model[word] for word in set(model.wv.vocab)])
num_clusters = word_vectors.shape[1] // 5

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print ("Time taken for K Means clustering: ", elapsed, "seconds.")

Time taken for K Means clustering:  56.54826641082764 seconds.


In [0]:
word_centroid_map = list(zip(model.wv.vocab, idx))

In [86]:
for cluster in range(50):
    #
    # Print the cluster number  
    print (f"\nCluster {cluster}")
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(len(word_centroid_map)):
        if( word_centroid_map[i][1] == cluster ):
            words.append(word_centroid_map[i][0])
    print (words)


Cluster 0
['on', 'why', 'bottom', 'what', 'stirring', 'results', 'food', 'belle', 'need', 'until', 'ripping', 'body', 'soldiers', 'trained', 'watch', 'montage', 'buying', 'wall', 'butts', 'certainty', 'predict', 'almost', 'week', 'themed', 'favorite', 'dale', 'element', 'gwynne', 'gradually', 'award', 'browsing', 'pros', 'skills', 'approval', 'kumar', 'fancy', 'ladies', 'forest', 'silver', 'slasher', 'anyone', 'cinematic', 'danny', 'horrible', 'restored', 'decision', 'jobs', 'disabled', 'seeking', 'current', 'world', 'unable', 'insane', 'poetry', 'davis', 'brent', 'crypt', 'columbo', 'disappointed', 'gorgeous', 'cushing', 'manages', 'suit', 'magical', 'suggestion', 'ho', 'needed', 'bed', 'toll', 'mix', 'division', 'palette', 'besides', 'extravagant', 'opens', 'untimely', 'liven', 'peace', 'rules', 'kung', 'surely', 'predator', 'email', 'neighborhood', 'shy', 'likeable', 'manson', 'pounds', 'characterized', 'guilt', 'committing', 'tokyo', 'comments', 'bitten', 'bravely', 'details', 'ex